In [12]:
# imports
import spotipy
import spotipy.util as util
import types
import requests
import json
import csv
import pandas as pd

In [2]:
# authorization
token = util.prompt_for_user_token(
        username="rosey.waters@gmail.com",
        scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing",
        client_id="e7b1f1096c82471d89d1db8030b82dd9",
        client_secret="5e635c87338b4729bc9a8bd007c2cbe1",
        redirect_uri="https://www.google.it/")

spotify = spotipy.Spotify(auth=token)

In [3]:
#This pulls the first 50 categories from the browse section of spotify, we'll use this information 
#to find the category ids, which will then pull playlist ids. 
cateogies_url = "https://api.spotify.com/v1/browse/categories"
def get_category_id(self, limit=1):
    return self._get(cateogies_url, limit=limit)

spotify.get_category_id = types.MethodType(get_category_id, spotify)

categories = spotify.get_category_id(limit=50)

data = json.dumps(categories, sort_keys=True, indent=2)

data_parse = json.loads(data)

categories = data_parse['categories']

In [4]:
#get the category keys
cat_keys = categories.keys()
print(cat_keys)

#from messing around with the data, I know that the important features will all be found in the "items" dictionary
print(categories['items'][0])

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
{'href': 'https://api.spotify.com/v1/browse/categories/toplists', 'icons': [{'height': 275, 'url': 'https://t.scdn.co/media/derived/toplists_11160599e6a04ac5d6f2757f5511778f_0_0_275_275.jpg', 'width': 275}], 'id': 'toplists', 'name': 'Top Lists'}


In [5]:
#Creating a list of category ids
cat_ids = []

for i in categories['items']:
    cat_ids.append(i['id'])
    
print(cat_ids)

['toplists', '2020', 'holidays', 'hiphop', 'pop', 'country', 'workout', 'at_home', 'rock', 'latin', 'mood', 'rnb', 'jre_podcast', 'gaming', 'shows_with_music', 'focus', 'edm_dance', 'blackhistorymonth', 'chill', 'indie_alt', 'inspirational', 'decades', 'instrumental', 'alternative', 'wellness', 'in_the_car', 'pride', 'party', 'sleep', 'classical', 'jazz', 'roots', 'soul', 'sessions', 'dinner', 'romance', 'kpop', 'punk', 'regional_mexican', 'popculture', 'blues', 'arab', 'desi', 'radar', 'anime', 'thirdparty', 'afro', 'comedy', 'metal', 'caribbean']


In [22]:
#Now there's a list of category ids, we can move on to getting playlists from them
playlist_info = []
for i in cat_ids:
    cat_url = 'https://api.spotify.com/v1/browse/categories/' + i + '/playlists'
    def get_playlist_ids(self, limit=1):
        return self._get(cat_url, limit=limit)

    spotify.get_playlist_ids = types.MethodType(get_playlist_ids, spotify)

    playlist_ids = spotify.get_playlist_ids(limit=50)

    data = json.dumps(playlist_ids, sort_keys=True, indent=2)

    data_parse = json.loads(data)

    playlists = data_parse['playlists']
    
    #let's get the playlist data now
    for i in playlists['items']:
        play = []
        play.append(i['description'])
        play.append(i['id'])
        play.append(i['name'])
        play.append(i['tracks']['total'])
        playlist_info.append(play)

In [23]:
print('Total number of playlists gathered: ' + str(len(playlist_info)))

Total number of playlists gathered: 1835


In [26]:
playlists = pd.DataFrame(data=playlist_info, columns = ['description','spotify_id','name','no_tracks']) 

In [27]:
playlists.head()

,description,spotify_id,name,no_tracks
0,Justin Bieber is on top of the Hottest 50!,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,50
1,"Music from Lil Durk, Playboi Carti and Kid Cudi.",37i9dQZF1DX0XUsuxWHRQd,RapCaviar,50
2,The newest music from Foo Fighters along with ...,37i9dQZF1DXcF6B6QPhFDv,Rock This,50
3,The world's biggest dance hits. Music from Ima...,37i9dQZF1DX4dyzvuaRJ0n,mint,79
4,"Today's top country hits of the week, worldwid...",37i9dQZF1DX1lVhptIYRda,Hot Country,53


In [28]:
playlists.to_csv('playlists.csv')